# Minimal NN to regress four-vector masses

This notebook is intended to find the minimal network structure that is needed to regress the mass of particle four-vectors, given either their
- energy and momentum compotents `(e, px, py, pz)`, or
- energy, transverse momentum as well as pseudo-rapidity and azimuthal angle `(e, pt, eta, phi)`.

The network is considered successful if at least 90% of previously unseen particle four-vectors are attributed a mass with an error of 10%.

Input four-vectors are randomly created for each batch following some pre-defined pdfs.

In [ ]:
# imports
import math
import functools

import tensorflow as tf
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import trange

## 1. Data generator

First we define generator of data. It's a simple function that returns a batch of four-vectors in the shape `(n, 4)`. You can control the allowed momentum and mass ranges, as well as the output basis which can be
- `cartesion=True` for `(e, px, py, pz)`, or
- `cartesion=False` for `(e, pt, eta, phi)`.

In [ ]:
# data generator
def get_data(n, min_p, max_p, min_m, max_m, cartesian=True):
    # uniform momentum
    p = np.random.uniform(min_p, max_p, n).astype(np.float32)
    # uniform masses
    m = np.random.uniform(min_m, max_m, n).astype(np.float32)
    # isotrop azimuth
    phi = np.random.uniform(-math.pi, math.pi, n).astype(np.float32)
    # centralized pseudo-rapidity with detector-like coverage
    eta = np.zeros_like(p)
    while np.mean(abs(eta) > 2.5) > 0.05:
        mask = abs(eta) > 2.5
        eta[mask] = np.random.normal(0, 2.5, np.sum(mask))
    eta = eta.astype(np.float32)
    # derive quantities
    e = (p * p + m * m)**0.5
    theta = 2.0 * np.arctan(np.exp(-eta))
    pt = p * np.cos(theta)
    if cartesian:
        px = pt * np.cos(phi)
        py = pt * np.sin(phi)
        pz = p * np.sin(theta)
        vec = [e, px, py, pz]
    else:
        vec = [e, pt, eta, phi]
    return np.stack(vec, axis=1), m

## 2. Neural network (model) definition

Next, we define a function that can generate our model. The implementation below aleady comes with a few bells and whistles, allowing you to configure the overall architecutre, activation functions and different [kernel / weight initializers](https://keras.io/api/layers/initializers/) as well as batch normalization, separate for the inut layer and all hidden layers.

In [ ]:
# model definition
def get_model(
    n_inputs,
    n_layers,
    n_units,
    activation="linear",
    kernel_initializer="glorot_uniform",
    batch_norm_first=False,
    batch_norm=False,
):
    x = tf.keras.layers.Input(shape=(n_inputs,))
    a = x

    # standard batch norm for input scaling
    if batch_norm_first:
        a = tf.keras.layers.BatchNormalization(axis=1)(a)
    
    # no batch norm when selu is chosen
    if activation == "selu":
        batch_norm = False
    
    # stack layers
    for _ in range(n_layers):
        # dense layer
        a = tf.keras.layers.Dense(n_units, kernel_initializer=kernel_initializer)(a)
        # batch norm before activation, except for relu
        if batch_norm and activation != "relu":
            a = tf.keras.layers.BatchNormalization(axis=1)(a)
        # activation
        a = tf.keras.layers.Activation(activation)(a)
        # batch norm after activation for relu
        if batch_norm and activation == "relu":
            a = tf.keras.layers.BatchNormalization(axis=1)(a)

    y = tf.keras.layers.Dense(1, activation="linear", kernel_initializer="glorot_normal")(a)
    model = tf.keras.Model(inputs=x, outputs=y)

    return model

## 3. Setup

Now it's time to initialize the model, define the loss (a standard mean squared error loss), and configure the function that will generate our input data batches (`functool.partial` allows to pin-point certain arguments of a function).

Also, we can define the function that should scale (and re-scale) the output target value to an eaiser-to-digest (for the network) range.

In [ ]:
# create the model, the loss function and the optimizer
model = get_model(n_inputs=4, n_layers=2, n_units=8)

# loss definition
@tf.function
def loss_func(pred, truth):
    mse = (pred - truth)**2.0
    # reduce to average over batch axis
    return tf.reduce_mean(mse, axis=0)

# define the function that creates a batch with configured values
data_spec = {"min_p": 0.0, "max_p": 200.0, "min_m": 0.1, "max_m": 100.0, "cartesian": True}
get_batch = functools.partial(get_data, **data_spec)

# function to (re)scale mass to usable (physical) range
target_mu = 0.5 * (data_spec["min_m"] + data_spec["max_m"])
target_sigma = (data_spec["max_m"] - data_spec["min_m"]) / 12.0**0.5

def scale_target(target, rescale=False):
    if rescale:
        return (target * target_sigma) + target_mu
    else:
        return (target - target_mu) / (target_sigma)

## 4. Single training step

For the training to run, we need to define what a single train step does.

For that, we define a function accepting the input `x`, the vector of true masses `truth`, and the instance of an `optimizer` that was configured before. With these ingredients, the train step itself is rather simple:

1. Perform the forward pass to obtain the network prediction.
2. Compare it to the truth in terms of the loss function.
3. Guard steps 1. and 2. by a `tf.GradientTape` that tracks gradients in all operations that are performed within its context.
4. From that tape, get the gradients of the loss with respect to all trainable variables.
5. Tell the optimizer to perform the weight update given the gradients.


In [ ]:
# training step
def train_step(x, truth, optimizer):
    with tf.GradientTape() as tape:
        # get prediction
        pred = model(x, training=True)[:, 0]
        # get loss
        loss = tf.reduce_mean(loss_func(pred, truth))
    # get and propagate gradients
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

## 5. Run the fulll training loop

All ingredients have be set up and configured at this point, so it's time to define and run our main training loop. It's supposed to do multiple things at once, though in a well defined order:

1. Configure and create an optimizer. We are going to use Adam.
2. We peform several iterations of
  - creating a new batch of input features, and corresponding truth values, and
  - perform a training step with the function we defined above.
3. After the training iterations, we create a larger batch and run the model prediction for it, so that we can
  - compute our success metric, i.e., the fraction of samples with a prediction error of less than 10%, and
  - create a few plots to get insights on the model performance.

In [ ]:
# main training loop
def train_loop(n_iterations, learning_rate=0.003):
    # optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate)

    # run train steps
    steps = trange(n_iterations)
    for i in steps:
        # get a batch of input vectors and true masses
        vec, truth = get_batch(n=64)

        # do a train step with the scaled mass
        loss = train_step(vec, scale_target(truth), optimizer)
        steps.set_postfix({"loss_mse": round(float(loss), 4)})
    
    # perform a test with more data
    vec, truth = get_batch(n=20000)
    pred_raw = model(vec, training=False)[:, 0]
    pred = scale_target(pred_raw, rescale=True)
    
    # relative error
    rel_err = (pred - truth) / truth
    print(f"predictions with error < 10%: {100 * np.mean(abs(rel_err) < 0.10):.2f}%")

    # 1d plot or relative error
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 5))
    ax1.hist(rel_err, bins=40, range=[-0.5, 0.5])
    ax1.set_xlim(-0.5, 0.5)
    ax1.set_xlabel("Relative error")
    ax1.set_ylabel("Entries")

    # 2d plot vs. E
    ax2.hist2d(rel_err, vec[:, 0], bins=[20, 20], range=[[-0.5, 0.5], [0, 200.0]])
    ax2.set_xlim(-0.5, 0.5)
    ax2.set_xlabel("Relative error")
    ax2.set_ylabel("E")

    # 2d plot vs. gamma
    gamma = vec[:, 0] / truth
    ax3.hist2d(rel_err, gamma, bins=[20, 20], range=[[-0.5, 0.5], [1, 7]])
    ax3.set_xlim(-0.5, 0.5)
    ax3.set_xlabel("Relative error")
    ax3.set_ylabel(r"$\gamma$ (E / m)")

    return pred, truth

In [ ]:
# a couple quick runs with large learning rates
pred, truth = train_loop(300, 0.03)
pred, truth = train_loop(300, 0.003)

In [ ]:
# longer training with a smaller rate
pred, truth = train_loop(2000, 0.001)

How did your network perform?
Try to improve it 👾

In [ ]:
... ❗️